In [14]:
import sys
import os # Make sure os is imported
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt

# Add the cloned repository directory to the Python path FIRST
repo_path = '/Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/'

print(f"Current Working Directory: {os.getcwd()}")
print(f"Attempting to use repo_path: {repo_path}")

# Check if the repo path exists
if not os.path.isdir(repo_path):
    print(f"ERROR: repo_path '{repo_path}' does not exist or is not a directory. Please verify the path.")
else:
    print(f"Contents of repo_path ('{repo_path}'):")
    try:
        for item in os.listdir(repo_path):
            print(f"  - {item}")
    except Exception as e:
        print(f"    Could not list contents: {e}")

print(f"\nOriginal sys.path (first 5 entries): {sys.path[:5]}...")

# Add to sys.path BEFORE importing
if repo_path not in sys.path:
    sys.path.append(repo_path)
    print(f"Appended to sys.path: {repo_path}")
else:
    print(f"{repo_path} already in sys.path.")
    
print(f"Updated sys.path (first 5 entries): {sys.path[:5]}...")

# NOW you can import from the yolo.py and other modules in the repo
try:
    from yolo import YOLO_np
    from common.utils import get_classes, get_anchors, get_colors
    from common.data_utils import preprocess_image
    print("Successfully imported modules from repo_path (yolo, common.utils, common.data_utils).")
except ModuleNotFoundError as e:
    print(f"ERROR: Could not import modules from repo_path. ModuleNotFoundError: {e}")
    print("Please ensure train.py, yolo.py, and folders like 'common', 'yolo3' are directly inside the repo_path listed above.")

print("\nTensorFlow version:", tf.__version__)

Current Working Directory: /Users/teaguesangster/Code/Python/ComputerVisionFinal
Attempting to use repo_path: /Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/
Contents of repo_path ('/Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/'):
  - tools
  - yolo4
  - yolo3
  - .DS_Store
  - LICENSE
  - requirements.txt
  - yolo2
  - data_for_yolo_val.txt
  - data_for_yolo_training.txt
  - Dockerfile
  - example
  - logs_finetune
  - weights
  - data-for-yolo-training-all-cut.txt
  - start.ps1
  - to_run.md
  - __pycache__
  - data-for-yolo-training-varroc.txt
  - cfg
  - README.md
  - common
  - .gitignore
  - configs
  - train.py
  - inference
  - yolo.py
  - eval.py
  - .git
  - assets
  - kitti_train_list_for_keras_yolo.txt
  - data_eval_test.txt

Original sys.path (first 5 entries): ['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/he

Annotation File for loading weights and the values we want to use -> 

In [15]:
# Loading our model ->
# These paths are relative to the repository root (repo_path) or absolute.
# Refer to `to_run.md` or `README.md` for typical paths for yolo3_xception
# and the pre-trained model details.

# Assuming you placed the downloaded dumped model in 'weights/' directory
weights_file_name = 'trained_final_dumped.h5' # Use the actual name of the downloaded dumped file
weights_path = os.path.join(repo_path, 'weights', weights_file_name)

# These config files should be in the 'configs/' directory of the cloned repo
anchors_path = os.path.join(repo_path, 'configs', 'yolo3_anchors.txt')

# The README mentions 'kitty_car_truck.txt' for dumping and 'kitty_all_except_nodata.txt' for evaluation.
# Let's use the one appropriate for the pretrained model's output.
# The pretrained model was trained on "Kitty dataset mentioned above" (all_except_nodata).
classes_path = os.path.join(repo_path, 'configs', 'kitty_all_except_nodata.txt')

# Model parameters (from README/to_run.md for yolo3_xception)
model_type = 'yolo3_xception' # Important, as this model is specifically for distance
model_image_size_str = '608x608'
height, width = model_image_size_str.split('x')
model_image_size = (int(height), int(width))

# Other parameters from yolo.py's default_config or typical usage
score_threshold = 0.1
iou_threshold = 0.4
elim_grid_sense = False # Default in yolo.py's config

# Prepare arguments for the YOLO class constructor (matching yolo.py)
# yolo_args = {
#     "model_type": model_type,
#     "weights_path": weights_path,
#     "anchors_path": anchors_path,
#     "classes_path": classes_path,
#     "score": score_threshold,
#     "iou": iou_threshold,
#     "model_image_size": model_image_size,
#     "elim_grid_sense": elim_grid_sense,
#     # "gpu_num": 1 # yolo.py default_config has this
# }

# In Cell 2, example for weights_path:
weights_path = os.path.join(repo_path, 'weights', 'trained_final_original.h5') # Ensure this file exists

yolo_args = {
    "model_type": 'yolo3_xception',
    "weights_path": weights_path,
    "anchors_path": os.path.join(repo_path, 'configs', 'yolo3_anchors.txt'),
    "classes_path": os.path.join(repo_path, 'configs', 'kitty_all_except_nodata.txt'),
    "score" : 0.1, # From yolo.py default_config
    "iou" : 0.4,   # From yolo.py default_config
    "model_image_size" : (608, 608), # From yolo.py default_config
    "elim_grid_sense": False, # From yolo.py default_config
    
    # "gpu_num" : 1, # yolo.py default_config has this, passed to YOLO_np constructor
    
}
# In Cell 2 (example)
yolo_args['annotation_file'] = "/Users/teaguesangster/Code/Python/ComputerVisionFinal/TrainingData/training/label_2/" # Replace with your actual file path
# Verify paths exist
if not os.path.exists(weights_path):
    print(f"ERROR: Weights file not found at {weights_path}. Please download and place it correctly.")
if not os.path.exists(anchors_path):
    print(f"ERROR: Anchors file not found at {anchors_path}.")
if not os.path.exists(classes_path):
    print(f"ERROR: Classes file not found at {classes_path}.")

Adding our Box Drawing methods:
This will allow us to see what we are scanning / recognizing -> 

In [16]:
# Creating a draw boxes function so we can see if we are outlining correctly ->
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt # Make sure matplotlib is imported for showing the image

def draw_boxes_simple(image_pil, boxes, classes, scores, class_names, colors, distances=None):
    """
    Draws bounding boxes, class names, scores, and optional distances on a PIL image.

    Args:
        image_pil (PIL.Image.Image): The image to draw on.
        boxes (np.ndarray): Array of bounding boxes, expected format [top, left, bottom, right].
        classes (np.ndarray): Array of class indices for each box.
        scores (np.ndarray): Array of confidence scores for each box.
        class_names (list): List of strings, mapping class index to class name.
        colors (list): List of RGB tuples for box colors, indexed by class index.
        distances (np.ndarray, optional): Array of distance values for each box.
    """
    if not isinstance(image_pil, Image.Image): # Ensure it's a PIL Image
        # If it's a NumPy array, convert it (assuming it's in RGB order)
        if isinstance(image_pil, np.ndarray):
            image_pil = Image.fromarray(image_pil.astype(np.uint8))
        else:
            # Handle other types or raise an error if necessary
            print("Warning: draw_boxes_simple received an unexpected image type.")
            return image_pil # Or raise error
        
    draw = ImageDraw.Draw(image_pil)
    try:
        # Try to load a common font, adjust size as needed
        font = ImageFont.truetype("arial.ttf", 15)
    except IOError:
        font = ImageFont.load_default() # Fallback to default font

    for i in range(len(boxes)): # Iterate through detected boxes
        if i >= len(classes) or i >= len(scores):
            print(f"Warning: Skipping box {i} due to missing class or score.")
            continue

        class_index = classes[i]
        if class_index >= len(class_names):
            print(f"Warning: Skipping box {i} due to invalid class_index {class_index} (max is {len(class_names)-1}).")
            continue
            
        predicted_class = class_names[class_index]
        box = boxes[i] 
        score = scores[i]
        
        # The 'boxes' from this YOLO model are typically [top, left, bottom, right]
        # after post-processing by functions like yolo3_postprocess_np.
        # Let's ensure they are integers for drawing.
        top, left, bottom, right = map(int, box) # Convert to int
        
        # Ensure coordinates are within image bounds
        img_width, img_height = image_pil.size
        top = max(0, top)
        left = max(0, left)
        bottom = min(img_height, bottom)
        right = min(img_width, right)
        
        if top >= bottom or left >= right:
            print(f"Warning: Skipping invalid box {i} with coordinates top={top}, left={left}, bottom={bottom}, right={right}")
            continue
            
        label_text = f'{predicted_class} {score:.2f}'
        if distances is not None and i < len(distances) and distances[i] is not None:
            # Assuming distances[i] is a single float value or a 1-element array
            dist_val = distances[i]
            if isinstance(dist_val, (np.ndarray, list)) and len(dist_val) > 0:
                dist_val = dist_val[0] # Take the first element if it's an array/list
            try:
                label_text += f' D: {float(dist_val):.1f}m' # Assuming distance is in meters
            except (ValueError, TypeError):
                 label_text += f' D: N/A'


        # Get text size using textbbox for potentially more accurate sizing
        text_bbox = draw.textbbox((0,0), label_text, font=font)
        label_width = text_bbox[2] - text_bbox[0]
        label_height = text_bbox[3] - text_bbox[1]

        text_origin_y = top - label_height - 2 # Place text above the box
        if text_origin_y < 0: # If it goes off the top, place it inside at the top
            text_origin_y = top + 2
        
        text_origin_x = left + 2

        # Pick color for the class
        color_tuple = colors[class_index % len(colors)] # Use modulo for safety

        # Draw rectangle for the bounding box
        for thickness_offset in range(2): # Draw a slightly thicker box
             draw.rectangle(
                 [left + thickness_offset, top + thickness_offset, right - thickness_offset, bottom - thickness_offset],
                 outline=color_tuple
             )
        
        # Draw background for the text label
        draw.rectangle(
            [text_origin_x, text_origin_y, text_origin_x + label_width + 4, text_origin_y + label_height + 2], # Added padding
            fill=color_tuple
        )
        # Draw the text
        draw.text((text_origin_x + 2, text_origin_y), label_text, fill=(0,0,0), font=font) # Black text

    del draw # Release the drawing context
    return image_pil

Splitting our Training and Validation Set -> 

In [17]:
# K.set_learning_phase(0) # yolo.py does this, K is tf.keras.backend
"""
# Check if weights file exists before attempting to load
if os.path.exists(yolo_args["weights_path"]):
    print(f"Loading model with weights from: {yolo_args['weights_path']}")
    yolo_model = YOLO(**yolo_args)
    print("Model loaded successfully.")
else:
    print(f"Model weights not found at {yolo_args['weights_path']}. Cannot load model.")
    yolo_model = None
    
    """
    
    # In Cell 3
if os.path.exists(yolo_args["weights_path"]): # yolo_args should now point to the original .h5
    print(f"Loading model with YOLO_np using weights: {yolo_args['weights_path']}")
    # The YOLO_np class constructor is similar
    yolo_model_np = YOLO_np(**yolo_args)
    print("YOLO_np model components loaded successfully.")
else:
    print(f"Original model weights not found at {yolo_args['weights_path']}. Cannot load YOLO_np model.")
    yolo_model_np = None
    
    

Loading model with YOLO_np using weights: /Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/weights/trained_final_original.h5


/opt/anaconda3/envs/keras-yolo3/lib/python3.8/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


backbone layers number: 132
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 608, 608, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 303, 303, 32)         864       ['image_input[0][0]']         
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 303, 303, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 303, 303, 32)         0     

Checking our GPU runtime -> 

In [18]:

import tensorflow as tf
import sys
import os

print("Checking GPU for training (from training cell)...")
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"Training will attempt to use GPU: {gpus[0].name}")
    
    # Configure GPU memory (important for stability)
    try:
        # Restrict memory growth to avoid GPU memory issues
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(f"Memory growth setting failed: {e}")
    
    # Set GPU as the default device
    gpus_logical = tf.config.list_logical_devices('GPU')
    
    # Test GPU computation
    print("\nTesting GPU computation...")
    try:
        with tf.device('/GPU:0'):  # Explicitly use the first GPU
            a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
            b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
            c = tf.matmul(a, b)
            print(f"GPU computation successful: {c.numpy()}")
    except RuntimeError as e:
        print(f"GPU test computation error: {e}")
        print("Falling back to CPU...")
else:
    print("WARNING: No GPU found by TensorFlow! Training will be very slow on CPU.")

Checking GPU for training (from training cell)...
Training will attempt to use GPU: /physical_device:GPU:0

Testing GPU computation...
GPU computation successful: [[22. 28.]
 [49. 64.]]


Annotating our file

In [19]:
import os
import numpy as np

def parse_annotation_file(annotation_path, dataset_base_path=""):
    """
    Parses the annotation file.
    Each line is expected to be: image_path x1,y1,x2,y2,class_id,distance ...
    The distance is the 6th element in each box_info block.

    Args:
        annotation_path (str): Path to the annotation text file.
        dataset_base_path (str, optional): A base path to prepend to image paths
                                           if they are relative in the annotation file.
                                           If image paths in the annotation file are absolute,
                                           or can be resolved directly, this can be empty.

    Returns:
        dict: A dictionary where keys are image filenames (or full paths as in file)
              and values are lists of ground truth objects.
              Each ground truth object is a dict:
              {'box': [xmin, ymin, xmax, ymax], 'class_id': int, 'distance': float}
    """
    ground_truth_data = {}
    if not os.path.exists(annotation_path):
        print(f"Error: Annotation file not found at {annotation_path}")
        return ground_truth_data

    with open(annotation_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            
            image_file_info = parts[0]
            # Construct the key for the dictionary.
            # Using basename allows matching even if dataset_base_path is different from actual test image path structure,
            # as long as filenames are unique.
            image_key = os.path.basename(image_file_info) # Use filename as key for easier lookup later

            gt_objects_for_image = []
            for i in range(1, len(parts)):
                try:
                    box_info_str = parts[i].split(',')
                    if len(box_info_str) == 6: # xmin,ymin,xmax,ymax,class_id,distance
                        xmin = int(box_info_str[0])
                        ymin = int(box_info_str[1])
                        xmax = int(box_info_str[2])
                        ymax = int(box_info_str[3])
                        class_id = int(box_info_str[4])
                        distance = float(box_info_str[5]) # Distance is the 6th element
                        
                        gt_objects_for_image.append({
                            'box': [xmin, ymin, xmax, ymax], # Standard YOLO box format
                            'class_id': class_id,
                            'distance': distance
                        })
                    elif len(box_info_str) == 5: # xmin,ymin,xmax,ymax,class_id (older format without distance)
                         print(f"Warning: Box info for {image_key} has 5 elements (no distance): {parts[i]}")
                    else:
                        print(f"Warning: Malformed box info for {image_key}: {parts[i]}")
                except ValueError as e:
                    print(f"Warning: Could not parse box_info '{parts[i]}' for image {image_key}. Error: {e}")
            
            if gt_objects_for_image:
                ground_truth_data[image_key] = gt_objects_for_image
            #else:
                #print(f"Debug: No valid ground truth objects found for image line: {line.strip()}")


    print(f"Parsed {len(ground_truth_data)} images from annotation file: {annotation_path}")
    return ground_truth_data

Labeling our test Images ->

In [20]:
# K.set_learning_phase(0) # yolo.py does this, K is tf.keras.backend
"""
# Check if weights file exists before attempting to load
if os.path.exists(yolo_args["weights_path"]):
    print(f"Loading model with weights from: {yolo_args['weights_path']}")
    yolo_model = YOLO(**yolo_args)
    print("Model loaded successfully.")
else:
    print(f"Model weights not found at {yolo_args['weights_path']}. Cannot load model.")
    yolo_model = None
    
    """
    
    # In Cell 3
if os.path.exists(yolo_args["weights_path"]): # yolo_args should now point to the original .h5
    print(f"Loading model with YOLO_np using weights: {yolo_args['weights_path']}")
    # The YOLO_np class constructor is similar
    yolo_model_np = YOLO_np(**yolo_args)
    print("YOLO_np model components loaded successfully.")
else:
    print(f"Original model weights not found at {yolo_args['weights_path']}. Cannot load YOLO_np model.")
    yolo_model_np = None
    
    

Loading model with YOLO_np using weights: /Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/weights/trained_final_original.h5
backbone layers number: 132
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 608, 608, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 303, 303, 32)         864       ['image_input[0][0]']         
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 303, 303, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       


In [21]:
# import tensorflow as tf
# import sys # Import sys for platform check
# 
# print("TensorFlow version:", tf.__version__)
# print("Python version:", sys.version)
# print("Platform:", sys.platform)
# 
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     print(f"Found {len(gpus)} Physical GPU(s):")
#     for i, gpu in enumerate(gpus):
#         print(f"  GPU {i}: Name: {gpu.name}, Type: {gpu.device_type}")
#         try:
#             tf.config.experimental.set_memory_growth(gpu, True)
#             print(f"    Memory growth enabled for {gpu.name}")
#         except RuntimeError as e:
#             # Memory growth can only be set once at program startup.
#             print(f"    Could not set memory growth for {gpu.name}: {e}")
# 
#     # A simpler way to test if GPU is usable for computation
#     # without explicitly using tf.device with the problematic string
#     try:
#         # Create a simple model and try to run it.
#         # TensorFlow should automatically place it on the GPU if available and configured.
#         model_test = tf.keras.Sequential([
#             tf.keras.layers.Dense(10, activation='relu', input_shape=(None, 10)),
#             tf.keras.layers.Dense(1)
#         ])
#         model_test.compile(optimizer='adam', loss='mse')
#         dummy_data = tf.random.normal((1,10))
#         dummy_labels = tf.random.normal((1,1))
#         model_test.fit(dummy_data, dummy_labels, epochs=1, verbose=0)
#         print("GPU test computation (simple model fit) seems successful.")
#     except Exception as e:
#         print(f"GPU test computation (simple model fit) error: {e}")
#         import traceback
#         traceback.print_exc()
# 
# else:
#     print("  No GPU found by TensorFlow. Ensure tensorflow-metal is correctly installed and compatible for your macOS and Python version.")
#     if sys.platform == "darwin": # macOS
#         print("  For M1/M2/M3 Macs, you need 'tensorflow-metal'. Install with: pip install tensorflow-metal")

Creating a testing function to see if our model is detecting correctly ->


Creating a cell for labeling the TestImages


Creating a methdod to parse all of our labels so it matches this dataset -> 

In [22]:

# This function should ideally be in a cell before Cell 5, or at the top of Cell 5
def load_kitti_ground_truth_for_image(label_file_path, model_class_names_list):
    """
    Parses a single KITTI label file.
    Each line is: ClassName truncated occluded alpha xmin ymin xmax ymax h w l x y z rot_y score
    Distance is 'z' (14th element, index 13).
    Box is xmin, ymin, xmax, ymax (indices 4, 5, 6, 7).

    Args:
        label_file_path (str): Path to the KITTI label .txt file.
        model_class_names_list (list): List of class names the Keras model uses,
                                     where index corresponds to class_id.

    Returns:
        list: A list of ground truth objects for the image.
              Each object is a dict:
              {'box': [xmin, ymin, xmax, ymax], 'class_id': int, 'distance': float, 'class_name': str}
              Returns empty list if file not found or no valid objects.
    """
    gt_objects_for_image = []
    if not os.path.exists(label_file_path):
        print(f"Warning: Label file not found at {label_file_path}")
        return gt_objects_for_image

    # Create a mapping from class name to the ID the model expects
    name_to_id_map = {name: i for i, name in enumerate(model_class_names_list)}

    with open(label_file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if not parts or len(parts) < 14: # Need at least 14 elements for distance
                continue

            class_name_gt = parts[0]
            if class_name_gt == 'DontCare' or class_name_gt not in name_to_id_map:
                continue # Skip 'DontCare' or classes not in model's list

            try:
                # KITTI format: Class truncated occluded alpha xmin ymin xmax ymax h w l x y z rot_y (score)
                # Indices:          0       1         2       3    4    5    6    7   8 9 10 11 12 13 14
                xmin = float(parts[4])
                ymin = float(parts[5])
                xmax = float(parts[6])
                ymax = float(parts[7])
                distance = float(parts[13]) # 'z' coordinate is distance
                
                # Map ground truth class name to the integer ID used by the model
                class_id_gt = name_to_id_map[class_name_gt]

                gt_objects_for_image.append({
                    'box': [xmin, ymin, xmax, ymax], # These are absolute pixel coordinates
                    'class_id': class_id_gt,
                    'class_name': class_name_gt,
                    'distance': distance
                })
            except ValueError as e:
                print(f"Warning: Could not parse line '{line.strip()}' in {label_file_path}. Error: {e}")
            except KeyError as e:
                print(f"Warning: Class name '{class_name_gt}' from label file not found in model's class list: {e}")
                
    return gt_objects_for_image

In [23]:
# --- Cell 5: Process Multiple Images with Ground Truth Distance Comparison ---
# Ensure these imports are present (usually in Cell 1 or earlier)
# import sys, os, numpy as np, tensorflow as tf (if not already done)
# from PIL import Image, ImageDraw, ImageFont
# import matplotlib.pyplot as plt
# from yolo import YOLO_np # Assuming using YOLO_np
# from common.utils import get_classes, get_anchors, get_colors
# from common.data_utils import preprocess_image
import math # For math.isnan

# Reminder:
# 1. 'draw_boxes_simple' function (robust version) should be defined in a previous cell or above.
# 2. 'load_kitti_ground_truth_for_image' should be defined in a previous cell or above.
# 3. 'yolo_model_np_instance' should be loaded (from Cell 3, using YOLO_np and "original" weights).
# 4. 'yolo_args' (from Cell 2) should contain 'classes_path'.
# 5. 'get_classes', 'get_colors', 'preprocess_image' should be imported (usually Cell 1).

if 'yolo_model_np_instance' in locals() and yolo_model_np_instance is not None and \
   'yolo_args' in locals() and 'classes_path' in yolo_args:

    # Load class names the model was trained on (needed for GT parsing and visualization)
    model_class_names = get_classes(yolo_args['classes_path'])
    model_colors = get_colors(model_class_names)

    # Define paths
    base_image_folder_path = "/Users/teaguesangster/Code/Python/ComputerVisionFinal/TrainingData/data_object_image_2/training/image_2/"
    base_label_folder_path = "/Users/teaguesangster/Code/Python/ComputerVisionFinal/TrainingData/training/label_2/"

    # Get list of available label files (and thus image files)
    # Let's use os.listdir on the label folder and take a slice for processing
    try:
        all_label_filenames = sorted([f for f in os.listdir(base_label_folder_path) if f.endswith('.txt')])
    except FileNotFoundError:
        print(f"ERROR: Ground truth label directory not found: {base_label_folder_path}")
        all_label_filenames = []

    num_images_to_process = 10 # Start with 10, you can increase to 100
    image_filenames_to_process = [f.replace('.txt', '.png') for f in all_label_filenames[:num_images_to_process]]


    for image_filename in image_filenames_to_process:
        test_image_path = os.path.join(base_image_folder_path, image_filename)
        corresponding_label_filename = image_filename.replace('.png', '.txt')
        test_label_path = os.path.join(base_label_folder_path, corresponding_label_filename)

        if not os.path.exists(test_image_path):
            print(f"ERROR: Test image not found at {test_image_path}. Skipping.")
            continue
        # Ground truth for this image will be loaded by load_kitti_ground_truth_for_image below

        print(f"\n--- Processing image: {test_image_path} ---")
        input_image_pil = Image.open(test_image_path)
        original_image_width, original_image_height = input_image_pil.size

        image_data_processed = preprocess_image(input_image_pil, yolo_model_np_instance.model_image_size)
        original_image_shape_tuple = (original_image_height, original_image_width)

        print(f"Preprocessed image data shape: {image_data_processed.shape}")
        print(f"Original image shape for model: {original_image_shape_tuple}")
        
        # Load Ground Truth for this specific image
        current_gt_objects = load_kitti_ground_truth_for_image(test_label_path, model_class_names)
        if current_gt_objects:
            print(f"Ground Truth for {image_filename}: Found {len(current_gt_objects)} objects.")
            for idx, gt_obj in enumerate(current_gt_objects[:3]): # Print details for first 3 GT objects
                print(f"  GT Obj {idx}: Class '{gt_obj['class_name']}' (ID {gt_obj['class_id']}), Box {gt_obj['box']}, Dist {gt_obj['distance']:.2f}m")
        else:
            print(f"Ground Truth for {image_filename}: No valid objects found or label file missing.")


        print("Running prediction with YOLO_np...")
        try:
            out_boxes, out_classes, out_scores, out_distances_pred = yolo_model_np_instance.predict(
                image_data_processed, 
                original_image_shape_tuple
            )
            
            num_detections = len(out_boxes) if out_boxes is not None else 0
            print(f"Found {num_detections} potential objects.")

            if num_detections > 0:
                print("Predictions (Box[T,L,B,R], ClassID, Score, Predicted Distance):")
                for i in range(num_detections):
                    pred_dist_val = "N/A"
                    if out_distances_pred is not None and i < len(out_distances_pred) and out_distances_pred[i] is not None:
                        current_pred_dist = out_distances_pred[i]
                        if isinstance(current_pred_dist, (np.ndarray, list)) and len(current_pred_dist) > 0:
                            current_pred_dist = current_pred_dist[0]
                        if isinstance(current_pred_dist, (int, float, np.number)) and not math.isnan(current_pred_dist):
                            pred_dist_val = f"{float(current_pred_dist):.2f}m"
                    
                    print(f"  Pred Obj {i}: Box {out_boxes[i].astype(int)}, Class {out_classes[i]} ({model_class_names[out_classes[i]]}), Score {out_scores[i]:.3f}, Dist {pred_dist_val}")

                    # Simple comparison: if one GT and one Pred of same class, show diff
                    if len(current_gt_objects) == 1 and num_detections == 1 and out_classes[i] == current_gt_objects[0]['class_id']:
                        gt_dist = current_gt_objects[0]['distance']
                        if pred_dist_val != "N/A":
                            pred_dist_float = float(current_pred_dist) # Use the float version
                            diff = pred_dist_float - gt_dist
                            print(f"    Distance Comparison: Pred={pred_dist_float:.2f}m, GT={gt_dist:.2f}m, Difference={diff:+.2f}m")
                        else:
                            print(f"    Distance Comparison: Pred=N/A, GT={gt_dist:.2f}m")
            
            # Visualization
            result_image_pil = input_image_pil.copy()
            if num_detections > 0:
                result_image_pil = draw_boxes_simple(result_image_pil, 
                                                     out_boxes, 
                                                     out_classes, 
                                                     out_scores, 
                                                     model_class_names, # Use loaded model_class_names
                                                     model_colors,    # Use loaded model_colors
                                                     out_distances_pred)
            
            
            # ... (result_image_pil is created by draw_boxes_simple with predicted boxes)

            # Now, let's draw Ground Truth boxes on the SAME image for comparison
            if current_gt_objects: # current_gt_objects is loaded for each image
                draw_gt = ImageDraw.Draw(result_image_pil) # Draw on the image that already has predictions
                
                # Ensure model_class_names is available (loaded at the start of Cell 5)
                # Ensure model_colors is available (loaded at the start of Cell 5)

                for gt_idx, gt_obj in enumerate(current_gt_objects):
                    gt_box_kitti = gt_obj['box'] # This is [xmin, ymin, xmax, ymax] from KITTI label
                    gt_class_id = gt_obj['class_id']
                    gt_class_name = gt_obj['class_name'] # Added this in load_kitti_ground_truth_for_image
                    gt_distance = gt_obj['distance']

                    # Convert KITTI box [xmin, ymin, xmax, ymax] to what Pillow rectangle expects:
                    # (x0, y0, x1, y1) which is (left, top, right, bottom)
                    gt_left, gt_top, gt_right, gt_bottom = map(int, [gt_box_kitti[0], gt_box_kitti[1], gt_box_kitti[2], gt_box_kitti[3]])

                    # Clamp GT box to image dimensions (usually not needed if labels are good, but safe)
                    gt_left_clamped = max(0, gt_left)
                    gt_top_clamped = max(0, gt_top)
                    gt_right_clamped = min(original_image_width, gt_right)
                    gt_bottom_clamped = min(original_image_height, gt_bottom)

                    if gt_top_clamped >= gt_bottom_clamped or gt_left_clamped >= gt_right_clamped:
                        print(f"Note: Skipping invalid GT box {gt_idx} after clamping: L{gt_left_clamped}, T{gt_top_clamped}, R{gt_right_clamped}, B{gt_bottom_clamped}")
                        continue
                    
                    # Use a distinct color for GT boxes, e.g., green
                    gt_draw_color_tuple = (255, 0, 0) # Green

                    # Draw GT rectangle (you can make it thicker or dashed for more distinction)
                    for offset in range(2): # Draw a 2px thick box
                        draw_gt.rectangle(
                            [gt_left_clamped + offset, gt_top_clamped + offset, gt_right_clamped - offset, gt_bottom_clamped - offset],
                            outline=gt_draw_color_tuple
                        )
                    
                    # Add GT label
                    gt_label_text = f"GT: {gt_class_name} ({gt_distance:.1f}m)"
                    try:
                        # Using the same font settings as draw_boxes_simple for consistency
                        font = ImageFont.truetype("arial.ttf", 15)
                    except IOError:
                        font = ImageFont.load_default()
                    
                    gt_text_bbox = draw_gt.textbbox((0,0), gt_label_text, font=font)
                    gt_label_width = gt_text_bbox[2] - gt_text_bbox[0]
                    gt_label_height = gt_text_bbox[3] - gt_text_bbox[1]

                    gt_text_origin_y = gt_top_clamped # Place text inside, at the top of the GT box
                    if gt_text_origin_y + gt_label_height > original_image_height: # If it overflows, move it up
                         gt_text_origin_y = gt_top_clamped - gt_label_height - 2
                    
                    gt_text_origin_x = gt_left_clamped + 2

                    # Draw background for the GT text label
                    draw_gt.rectangle(
                        [gt_text_origin_x, gt_text_origin_y, gt_text_origin_x + gt_label_width + 4, gt_text_origin_y + gt_label_height + 2],
                        fill=gt_draw_color_tuple
                    )
                    draw_gt.text((gt_text_origin_x + 2, gt_text_origin_y), gt_label_text, fill=(0,0,0), font=font) # Black text

                del draw_gt # Release drawing context
            
            
            plt.figure(figsize=(14, 14))
            plt.imshow(result_image_pil)
            plt.title(f"Detections on {os.path.basename(test_image_path)}")
            plt.axis('off')
            plt.show()

        except Exception as e:
            print(f"An error occurred for {test_image_path}: {e}")
            import traceback
            traceback.print_exc()
        print("--- End processing for image ---")
else:
    print("Required variables (yolo_model_np_instance, yolo_args, or get_classes) not found. Please run previous cells.")

Required variables (yolo_model_np_instance, yolo_args, or get_classes) not found. Please run previous cells.


Time to start training!

In [24]:
import os
import numpy as np

# Paths from your setup
base_kitti_image_folder = "/Users/teaguesangster/Code/Python/ComputerVisionFinal/TrainingData/data_object_image_2/training/image_2/"
base_kitti_label_folder = "/Users/teaguesangster/Code/Python/ComputerVisionFinal/TrainingData/training/label_2/"
# This is the classes file your Keras model uses (from yolo_args in Cell 2)
# Ensure yolo_args and get_classes are available if running this cell standalone after kernel restart
# Or define classes_path_for_conversion directly:
classes_path_for_conversion = yolo_args['classes_path'] # Or replace with direct path string
# Output file for training
output_annotation_file = os.path.join(repo_path, "kitti_train_list_for_keras_yolo.txt") # repo_path from Cell 1

def create_aggregated_annotation_file(image_folder, label_folder, model_classes_file, output_file):
    print(f"Loading model class names from: {model_classes_file}")
    # Ensure get_classes is imported: from common.utils import get_classes
    model_class_names_list = get_classes(model_classes_file)
    name_to_id_map = {name: i for i, name in enumerate(model_class_names_list)}
    print(f"Model class map: {name_to_id_map}")

    aggregated_annotations = []
    
    label_filenames = sorted([f for f in os.listdir(label_folder) if f.endswith('.txt')])
    if not label_filenames:
        print(f"Error: No label files found in {label_folder}")
        return

    print(f"Found {len(label_filenames)} label files. Processing...")

    for label_filename in label_filenames:
        image_filename = label_filename.replace('.txt', '.png')
        image_file_path_abs = os.path.join(image_folder, image_filename) # Absolute path to image
        
        # The train.py script might expect paths relative to args.dataset_working_directory
        # or absolute paths. The example in README uses absolute paths within the string.
        # Let's use absolute paths for robustness.
        
        if not os.path.exists(image_file_path_abs):
            # print(f"Warning: Image file {image_file_path_abs} not found for label {label_filename}. Skipping.")
            continue

        kitti_label_file_path = os.path.join(label_folder, label_filename)
        line_parts = [image_file_path_abs] # Start with image path

        with open(kitti_label_file_path, 'r') as f_label:
            for kitti_line in f_label:
                obj_parts = kitti_line.strip().split()
                if not obj_parts or len(obj_parts) < 14:
                    continue

                class_name_gt = obj_parts[0]
                if class_name_gt == 'DontCare' or class_name_gt not in name_to_id_map:
                    continue
                
                try:
                    # KITTI format: ClassName trunc occ alpha xmin ymin xmax ymax h w l x y z rot_y (score)
                    xmin = float(obj_parts[4])
                    ymin = float(obj_parts[5])
                    xmax = float(obj_parts[6])
                    ymax = float(obj_parts[7])
                    distance = float(obj_parts[13]) # 'z' coordinate
                    class_id_model = name_to_id_map[class_name_gt]

                    # Format: x1,y1,x2,y2,cls,dist (no spaces in this part)
                    box_info_str = f"{int(xmin)},{int(ymin)},{int(xmax)},{int(ymax)},{class_id_model},{int(round(distance))}"
                    line_parts.append(box_info_str)
                except (ValueError, KeyError) as e:
                    print(f"Warning: Skipping object in {label_filename} due to parsing error: {obj_parts} | Error: {e}")
        
        if len(line_parts) > 1: # Only add if there are objects for this image
            aggregated_annotations.append(" ".join(line_parts))

    with open(output_file, 'w') as f_out:
        for line in aggregated_annotations:
            f_out.write(line + "\n")
    print(f"Aggregated annotation file created at: {output_file} with {len(aggregated_annotations)} image entries.")

# --- Call the function ---
# Make sure 'repo_path' and 'yolo_args' (containing 'classes_path') are defined from previous cells
if 'repo_path' in locals() and 'yolo_args' in locals() and 'get_classes' in globals():
    create_aggregated_annotation_file(base_kitti_image_folder, 
                                      base_kitti_label_folder, 
                                      yolo_args['classes_path'], 
                                      output_annotation_file)
else:
    print("Error: Prerequisite variables (repo_path, yolo_args, get_classes) not defined. Please run previous cells.")

Loading model class names from: /Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/configs/kitty_all_except_nodata.txt
Model class map: {'Pedestrian': 0, 'Car': 1, 'Van': 2, 'Truck': 3, 'Person_sitting': 4, 'Cyclist': 5, 'Tram': 6}
Found 7481 label files. Processing...
Aggregated annotation file created at: /Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/kitti_train_list_for_keras_yolo.txt with 7481 image entries.


Creating new Parameters as we wish to train our model for a few Passes -> 

In [25]:
import argparse # For creating an args namespace
import os # Make sure os is imported if not already in this cell from prior ones
# Ensure repo_path and output_annotation_file, and yolo_args are defined from previous cells

# --- Training Configuration ---
train_args = argparse.Namespace(
    model_type='yolo3_xception', # Crucial for distance model
    weights_path=os.path.join(repo_path, 'weights', 'trained_final_original.h5'), # START FROM THE PRE-TRAINED ORIGINAL MODEL
    annotation_file=output_annotation_file, # The aggregated file we just created
    anchors_path=os.path.join(repo_path, 'configs', 'yolo3_anchors.txt'),
    classes_path=yolo_args['classes_path'], # From your Cell 2
    model_image_size=(608, 608), # Must be tuple of ints
    batch_size=8,  # Keep low for M3 Mac unless you have a lot of VRAM, README suggests 2 or 3
    total_epoch=yolo_args.get('total_epoch_base', 43) + 20, # Train for 20 more epochs from current (e.g. 43+20=63)
                                                            # The README says pretrained model was for 43 epochs.
    init_epoch=yolo_args.get('total_epoch_base', 43), # Start from the epoch count of the loaded model
    learning_rate=1e-7,  # Start with a very LOW learning rate for fine-tuning
                         # README warns about high LR: "Use low learning rates (e.g., 1e-6) at the beginning...with random initial weights"
                         # Since we are fine-tuning, 1e-5 or 1e-4 might be okay.
    optimizer='adam',
    log_dir=os.path.join(repo_path, 'logs_finetune'), # New log directory for this fine-tuning
    checkpoint_period=1, # Save checkpoint every epoch
    val_split=0.1, # Use 10% of the kitti_train_list_for_keras_yolo.txt for validation during training
    val_annotation_file=None, # We'll use val_split from the main annotation_file
    freeze_level=0, # 0 to train all layers, 1 to freeze backbone. Start with 0 for fine-tuning all.
                   # The README example uses 0.
    transfer_epoch=0, # No separate transfer stage needed when loading full model weights
    multiscale=False, # As per README example command
    rescale_interval=10, # Default in train.py
    enhance_augment=None, # As per README example command
    label_smoothing=0.0, # Consistent with train.py default
    multi_anchor_assign=False, # Default in train.py
    elim_grid_sense=False, # Default
    data_shuffle=True, # Shuffle data
    gpu_num=1,
    model_pruning=False, # Consistent with train.py default
    eval_online=True, # Optional: evaluate on val set during training
    eval_epoch_interval=5, # Evaluate every 5 epochs
    save_eval_checkpoint=False, # Consistent with train.py default
    dataset_working_directory="", # Set this if your image paths in annotation_file are relative.
                                 # Since we wrote absolute paths, this can be empty.
    decay_type=None, # Consistent with train.py default

    # Callback related arguments defined in train.py's ArgumentParser:
    lr_patience=10, # Default from train.py
    min_lr=1e-10, # Default from train.py
    early_stopping_patience=50, # Default from train.py
    save_best_only=False,  # **** THIS WAS THE MISSING ARGUMENT ****
                           # Set to True if you only want the best checkpoint based on val_loss

    # Worker related arguments (add if you modify train.py to expect these, otherwise defaults in train.py's main() are used)
    # These are used inside model.fit, not directly by the argparser in train.py's `if __name__ == '__main__'`
    # but it's good practice if you were to pass them or if train.py was refactored.
    # For now, train.py uses its own defaults for these if not present in the args object directly.
    # The main `train.py`'s argparse *does* define these:
    workers=4, # Default from train.py
    use_multiprocessing=True, # Default from train.py
    max_queue_size=32 # Default from train.py
)

# Ensure log_dir exists
os.makedirs(train_args.log_dir, exist_ok=True)

print("Training arguments prepared:")
for arg, value in sorted(vars(train_args).items()):
    print(f"  {arg}: {value}")

Training arguments prepared:
  anchors_path: /Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/configs/yolo3_anchors.txt
  annotation_file: /Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/kitti_train_list_for_keras_yolo.txt
  batch_size: 8
  checkpoint_period: 1
  classes_path: /Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/configs/kitty_all_except_nodata.txt
  data_shuffle: True
  dataset_working_directory: 
  decay_type: None
  early_stopping_patience: 50
  elim_grid_sense: False
  enhance_augment: None
  eval_epoch_interval: 5
  eval_online: True
  freeze_level: 0
  gpu_num: 1
  init_epoch: 43
  label_smoothing: 0.0
  learning_rate: 1e-07
  log_dir: /Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/logs_finetune
  lr_patience: 10
  max_queue_size: 32
  min_lr: 1e-10
  model_image_size: (608, 608)
  model_pruning: False
  model_type:

And a cell for training our model ->

In [26]:
# --- Run Training ---
# Ensure train.py is accessible via sys.path (already done if repo_path is in sys.path)
# And that the 'train_args' object is defined from the cell above.

# It's good practice to ensure TensorFlow is using the GPU right before training
import tensorflow as tf
print("Checking GPU for training...")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Training will use GPU: {gpus[0].name}")
else:
    print("WARNING: No GPU found for training! Training will be very slow on CPU.")


# Import the main training function from train.py
# Make sure 'train' is not a conflicting variable name if you named a cell 'train'
from train import main as run_training_script # Renaming to avoid conflict

print(f"\nStarting fine-tuning for {train_args.total_epoch - train_args.init_epoch} more epochs...")
try:
    run_training_script(train_args)
    print("Fine-tuning completed.")
except Exception as e:
    print(f"An error occurred during training: {e}")
    import traceback
    traceback.print_exc()

Checking GPU for training...
Training will use GPU: /physical_device:GPU:0

Starting fine-tuning for 20 more epochs...


Using Yolo3DataGenerator (Sequence) for training data...
Using Yolo3DataGenerator (Sequence) for validation data...


backbone layers number: 132
backbone layers number: 132
Create  yolo3_xception model with 9 anchors and 7 classes.
model layer number: 199


Load weights /Users/teaguesangster/Code/Python/ComputerVisionFinal/ExistingModel/yolo-with-distance/weights/trained_final_original.h5.
Unfreeze all of the layers.
Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, None, None, 3)]      0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, None, None, 32)       864       ['image_input[0][0]']         
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, None, None, 32)       128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                            

Process Keras_worker_SpawnPoolWorker-35:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/keras-yolo3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/keras-yolo3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/keras-yolo3/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/keras-yolo3/lib/python3.8/multiprocessing/queues.py", line 355, in get
    with self._rlock:
  File "/opt/anaconda3/envs/keras-yolo3/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process Keras_worker_SpawnPoolWorker-33:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/keras-yolo3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/keras-yolo3/lib/python3.8/mu

KeyboardInterrupt: 